#  YOLOV3 NESNE TANIMA (OBJECT DETECTİON)
Bu notebookta nesne tanıma (object detection) için en hızlı modellerden biri olan yolov3 modeli ile nesne tanıma yapacağız.

1.ADIM = DARKNET KLONLAMA VE KURULUMU

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15460, done.
remote: Total 15460 (delta 0), reused 0 (delta 0), pack-reused 15460
Receiving objects: 100% (15460/15460), 14.05 MiB | 9.72 MiB/s, done.
Resolving deltas: 100% (10388/10388), done.


In [ ]:
#GPU ve opencv kullanabilmek için makefile dosyasında yapılması gereken değişiklikler.
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [ ]:
#Darknet Build
!make

## KENDİ NESNEMİZ İÇİN MODEL EĞİTİMİ
Şimdiye kadar yaptıklarımız daha önceden farklı nesneler için eğitilmiş modellerin ağırlıklarını kullanmaktı. 
Şimdi ise tamamen bizim nesne tanıma yapmak istediğimiz nesneler için eğitim yapacağız.

## ADIM 1: FOTOĞRAF TOPLAMA VE ETİKETLEME 
Üzerinde tanıma yapacağımız nesne için bir çok fotoğrafa ihtiyacımız olacak. Ne kadar fazla ve ne kadar farklı tip fotoğrafımız olursa o kadar doğru bir modelimiz olur. 

Fotoğraflarımızı 2 şekilde hazırlayabiliriz:

1)Google'ın sunmuş olduğu [Open İmages](https://storage.googleapis.com/openimages/web/index.html) sitesinde 600'ün üzerinde nesnenin etiketlenmiş halde binlerce fotoğrafı yer alıyor. Bu siteden istediğimiz kadar fotoğrafı indirip kullanabiliriz.(Open İmages kaynağından fotoğraf indirme için(https://www.youtube.com/watch?v=SjsF8_kDigw))

2)Tanıma yapacağımız nesneleri kendimiz fotoğraflayabilir veya google'dan bulduğumuz fotoğrafları etiketleyerek kullanabiliriz.(Kendi fotoğraflarımızı etiketlemek için(https://www.youtube.com/watch?v=H10L5BXdjdY&t=1s))

In [ ]:
#Öncelikle collabdan driveımıza bağlanmak için bir doğrulama yapmamız gerekiyor.
#Aşağıdaki kodu çalıştırarak çıkan linkte collab notebook'unu hangi hesabımız ile açtıysak o hesaba girerek gelen link ile doğrulama yapmalıyız.
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

/content
Mounted at /content/gdrive


In [ ]:
#Burada ise erişimi uzun olan drive dizini için kısa bir yol tanıma yapıyoruz
# "/content/gdrive/My\ Drive/" olan dizine /mydrive diyerek ulaşabiliriz.
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

 Classroom  'Colab Notebooks'   yolov3


## ADIM 2: HAZIRLAMIŞ OLDUĞUMUZ VERİ SETİNİ YÜKLEME
Hazırlamış olduğumuz veri setini google collab'e yüklemeliyiz.

Bunun için drive'ımızda bir yolov3 diye klasör oluşturalım.(bu klasörü daha sonra gerekli dosyaları bir arada toplamak için kullanacağız)

Veri setiniz bir klasör içine alıp zip haline getirerek drive'ımıza yükleyip oradan google collab içine aktarmak bize zaman kazandıracaktır.

In [ ]:
# oluşturduğumuz dosyayı kontrol edelim.
!ls /mydrive/yolov3

generate_train.py  obj.data  obj.names	obj.zip  yolov3_custom.cfg


In [ ]:
# Yüklemiş olduğumuz .zip dosyasını collaba kopyalayalım
%cd darknet/
!cp /mydrive/yolov3/obj.zip ../

/content/darknet


In [ ]:
# kopyaladığımız dosyayı zipten çıkaralım.
!unzip ../obj.zip -d data/

## ADIM 3: EĞİTİM İÇİN GEREKLİ DOSYALARI HAZIRLAYALIM

Bu adımda eğitim için gerekli olan .cfg file, obj.data, obj.names ve train.txt dosyalarını oluşturacağız.

*-Config Dosyası*

In [ ]:
#Öncelikle config dosyasını düzenleyelim.
#Darknet içindeki yolov3.cfg dosyasını drive'ımıza kopyalayalım ve adını değiştirelim.
!cp cfg/yolov3.cfg /mydrive/yolov3/yolov3_custom.cfg

Config dosyamızda yapmamız gereken değişiklikler:

(Burada verilen değerler bu değişkenlerin önerilen değerleridir.) 

1.   batch=1 ve subdivision = 1 satırlarını yorum satırı yapıyoruz.

2.   batch = 65 ve subdivision 16 satırlarını yorumdan çıkartıyoruz

3.   max_batches değerini (2000 * eğitilen sınıf sayısı) değerine eşitliyoruz.

4.   steps değerlerini (%80 of max_batches) , (%90 of max_batches) yapıyoruz.

5.   [yolo] başlığı altındaki classes değerlerini eğitim yaptığımız sınıf sayısı ile değiştiriyoruz. 

6.   filters değişkenlerini de (eğitim yapacağımız sınıf sayısı + 5 )*3 değerine eşitliyoruz 













In [ ]:
#Düzenlemiş olduğumuz config dosyasını geri yükleyelim
!cp /mydrive/yolov3/yolov3_custom.cfg ./cfg

*-obj.names ve obj.data*

yolov3 isimli klasörümüz içine obj.names isimli bir dosya oluşturalım ve dosyayı eğitim yapacağımız nesleriniz isimlerini yazıyoruz.

ÖRNEĞİN:
```
Guitar
Mobile Phone
```

Aynı klasör içinde obj.data isimle bir dosya oluşturarak içine eğitim yapacağımız nesne sayısını, eğitim yaparken kullanacağımız train.txt, text.txt ve obj.names isimli dosyaların adreslerini ve eğitim sonucu bulduğumuz ağırlıkları kaydedeceğimiz dizini yazıyoruz.

ÖRNEĞİN:
```
classes = 2
train = data/train.txt
valid = data/test.txt
names = data/obj.names
backup = /mydrive
```


In [ ]:
#Oluşturduğumuz obj.names ve obj.data dosyalarını google collaba aktaralım
!cp /mydrive/yolov3/obj.names ./data
!cp /mydrive/yolov3/obj.data  ./data

*-Train Dosyası*


generate_train.py



```
import os
image_files = []
os.chdir(os.path.join("data", "obj"))
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".jpg"):
        image_files.append("data/obj/" + filename)
os.chdir("..")
with open("train.txt", "w") as outfile:
    for image in image_files:
        outfile.write(image)
        outfile.write("\n")
    outfile.close()
os.chdir("..")
```



In [ ]:
#Train dosyamızı google collaba kopyalayalım
!cp /mydrive/yolov3/generate_train.py ./

In [ ]:
#Şimdi bu python dosyasını çalıştırarak train için gerekli olayları bizim için yapmasını bekleyebiliriz.
!python generate_train.py

Eğer bu adıma kadar sorunsuz bir şekilde geldiysek train doğru bir şekilde geldiysek train.txt dosyamızı kontrol edelim. 

Dosyamıza çift tıklayıp açtığımızda her satırda 1 eğitim fotoğrafı olacak şekilde düzenlendiğiniz görebiliriz.

# ADIM 4: ÖNCEDEN EĞİTİLMİŞ CONVOLUTİONAL KATMANLARIN AĞIRLIKLARINI İNDİRME

Bu adımda önceden eğitilmiş yolov3 için kullanılmış deeplearning katmanları ağırlıklarını indiriyoruz. Bu adımı uygulamak zorunda değiliz ama eğitime bu ağırlıklarla başlamak eğittiğimiz modelin daha doğru çalışmasına ve eğitim süresini kısaltmaya yardımcı olacaktır.

In [ ]:
#Ağırlıkları alalım
!wget http://pjreddie.com/media/files/darknet53.conv.74

--2022-07-27 13:41:32--  http://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://pjreddie.com/media/files/darknet53.conv.74 [following]
--2022-07-27 13:41:32--  https://pjreddie.com/media/files/darknet53.conv.74
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  18.5MB/s    in 9.4s    

2022-07-27 13:41:42 (16.5 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



# ADIM 5: KENDİ NESNE TANIYICIMIZI EĞİTELİM

Gerekli tüm dosyalar hazır, eğitime başlayabiliriz.

Eğitimimiz uzun süreceği için google collab bizi serverdan atabilir. Bunun önüne geçmek için aktif olduğumuzu bir şekile bildirmeliyiz.


Bunun için de sayfanın üst tarafına sağ tıklayıp "ögeyi denetle" veya "incele" seçeneğiniz seçip, çıkan pencereden "console"'a tıklayıp açılan komut satırına aşağıdaki kodu ekleyip enter tuşuna basarsak bu kod bizim 10 dakikada bir connect butonuna basarak bizim aktif kalmamızı sağlayacaktır.

```
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
```

# EĞİTİM

Sıradaki komut ile eğitim başlayacaktır.

Eğitimimizin süresi veri setinizdeki fotoğraf sayısı, fotoğrafların kalitesi, eğitim yaptığınız nesne sayısı gibi faktörlere göre değişebilir. Modelimizin doğruluğu için loss değerimiz önemlidir. Loss değerimiz ne kadar düşük olursa modelimiz o kadar doğru çalışır. Modelimizi loss değeri azalmayı durdurana kadar çalıştırıp veri setimize göre mümkün olan en doğru modeli eğitebiliriz. 



In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_custom.cfg  -dont_show

v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.801946), count: 4, class_loss = 0.639961, iou_loss = 0.228229, total_loss = 0.868189 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000020, iou_loss = 0.000000, total_loss = 0.000020 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000002, iou_loss = 0.000000, total_loss = 0.000002 
 total_bbox = 248642, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.852471), count: 4, class_loss = 0.752452, iou_loss = 0.113308, total_loss = 0.865761 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000921, iou_loss = 0.000000, total_loss = 0.000921 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_